In [1]:
import requests
from bs4 import BeautifulSoup
import csv

In [29]:
#url de libros Uptc
url = 'https://editorial.uptc.edu.co/catalogo.html'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html')


scrap=soup.find('div',class_="sa-product-container-v3 grid")
productos_url = scrap.find_all('li',class_="sa-product-item-v3 sa-product_grid_size")

url_libros = [url.find('a')["href"] for url in productos_url]

datos = {
        "title": None ,
        "colaboradores":None,
        "precio":None,
        "ano": None,
        "palabras clave":None,
        "doi":None,
        "descripcion":None,
    }




In [32]:
url_libros[-1]

'https://editorial.uptc.edu.co/gpd-la-leche-humana-un-amor-quimico-9789586608602-6688706a1f37c.html'

In [113]:
titulos = []
autores = []
palabras_clave = []
precios = []
doi = []
descripcion = []
url = url_libros[0]
res = requests.get(url)
soup = BeautifulSoup(res.text,'html')


scrap=soup.find('div',class_="product-info-main")
titulos.append(scrap.find('div',class_="col-xs-12").find('h2').text)
## busquemos lo autores
temp = [i for i in scrap.find_all('div',class_="col-xs-12")]
temp1 =[ a.text for a in temp[1].find_all('a')  if 'products_author' in a['href'] ]
autores.append(temp1)
# busca las palabras clave
temp1 =[ a.text for a in temp[1].find_all('a')  if 'products_keywords' in a['href'] ]
palabras_clave.append()
# DOi
doi.append([c[-1].text  for c in temp])
# precios
temp = [a for a in scrap.find_all('div',class_='price-box price-final_price')]
temp1 = [a.text.strip() for a in temp]
precios.append(temp1)
#descripcion
descripcion.append(scrap.find('div',class_='description-wrapper').text)







SyntaxError: invalid syntax (256830141.py, line 17)

In [169]:
scrap=soup.find('div',class_="filter-options panel-group")

In [181]:
scrap[]

KeyError: 0

In [184]:
scrap.find_all('strong')[0].text

'Detalle de formato:'

In [179]:
for a in scrap.find_all('span'):
    print(a.text)

eBook - EPUB
Digital (suministrado electrónicamente)
EPUB
Activo
9789586609272
Texto (legible a simple vista)
DRM
Share
Permitido, con limitaciones
Español
1,593 Kbytes
Universidad Pedagógica y Tecnológica de Colombia
Nueva edición
1
Tunja
Colombia
2025
Exclusivo para un punto o canal de venta
COP $30.000
Libro impreso - Tapa blanda o Bolsillo
Tapa blanda o Bolsillo
Activo
9789586609265
Texto (legible a simple vista)
Español
14 x 21  x 0.6
106 Páginas
universidad Pedagógica y Tecnológica de Colombia
Nueva edición
1
Tunja
Colombia
2025
Exclusivo para un punto o canal de venta
419307
COP $40.000


In [153]:
temp1

['', 'COP $40.000', 'COP $30.000']

In [115]:
temp1

['Fluido',
 'Sistema de bombeo',
 'Bomba centrífuga',
 'Mecánica de fluidos',
 'Transporte de fluidos',
 'Operación y Mantenimiento']

In [73]:
temp1

['Otto Caro Niño',
 'Orlando Díaz Parra',
 'Fluido',
 'Sistema de bombeo',
 'Bomba centrífuga',
 'Mecánica de fluidos',
 'Transporte de fluidos',
 'Operación y Mantenimiento',
 '10.19053/uptc.9789586609265']

In [9]:
res.status_code

200

 tenemos un estatus 200 lo cual es correcto la solicitud
 

In [4]:
soup = BeautifulSoup(res.text,'html')

In [19]:
soup.find('title').get_text()

'Catálogo - UPTC'

In [22]:
datos=soup.find('div',class_="sa-product-container-v3 grid")


In [26]:
productos = datos.find_all('li',class_="sa-product-item-v3 sa-product_grid_size")

In [ ]:
a = {
    titulo: 
    colaboradores:
    ano:
    precio:
}

In [48]:
title =[i.find('span').get_text(strip=True) for i in productos]

In [62]:
ano = [i.find('div',class_='product-list-edition').get_text(strip=True) for i in productos]

In [76]:
productos[].find('span',class_='minimal-price').get_text()

'\nImpreso\n\n\nCOP $40.000\n\n\n'

In [101]:
datos = []
for i in productos:
    a = {
        "title":i.find('span').get_text(strip=True),
        "colaboradores":list(i.find('div',class_='sa-product-author-v3').stripped_strings)[1],
        "precio": i.find('span',class_='minimal-price').get_text().replace("\n",""),
        "ano": i.find('div',class_='product-list-edition').get_text(strip=True)
    }
    datos.append(a)


In [102]:
datos

[{'title': 'Transporte de fluidos bombas centrífugas',
  'colaboradores': 'Otto Caro Niño                                                                                     y otros',
  'precio': 'eBookCOP $30.000',
  'ano': '2025'},
 {'title': 'Introducción al cálculo de probabilidades para estudiantes de ingeniería',
  'colaboradores': 'Alex L. Rojas',
  'precio': 'eBookGratuito',
  'ano': '2025'},
 {'title': 'Derechos colectivos en Colombia:',
  'colaboradores': 'Luis Bernardo Díaz Gamboa                                                                                     y otros',
  'precio': 'eBookGratuito',
  'ano': '2025'},
 {'title': 'Resistencias y voces de las mujeres:',
  'colaboradores': 'Celina de Jesús Trimiño Velásquez                                                                                     y otros',
  'precio': 'ImpresoCOP $40.000',
  'ano': '2025'},
 {'title': 'Manual de prácticas pedagógicas para la formación de maestros.',
  'colaboradores': 'Myriam Cecilia

## 1.2 Encontrando las url de cada producto.

+ esto nos permitira acceder a toda la informacion del producto 

In [ ]:
productos = soup.find('div',class_='sa-product-container-v3 grid').find_all('li')
urls=[url.find('a') ['href'] for url in productos]

In [18]:
len(productos)

68

In [31]:
productos[0].find('a')

<a href="https://editorial.uptc.edu.co/gpd-transporte-de-fluidos-bombas-centrifugas-9789586609265-67ebeccc7db73.html">
<img alt="Portada de la publicación Transporte de fluidos bombas centrífugas" src="https://simehbucket.s3.amazonaws.com/images/415ebe2115322046c17eea71821248a5-medium.jpg"/>
</a>